In [1]:
from datetime import datetime,date
import time
import sys
sys.path.append(r'/home/hengtian/lucy/factor-analyzer-v4/')
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import multiprocessing
import os
import queue
import tqdm
# from src import data 
# from src import analyzer
sys.path.append(os.path.abspath('/home/hengtian/kate/'))
import get_factor_tools as gft
import datetime as dt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# import cufflinks as cf
# cf.go_offline()

sys.path.append(r'/home/hengtian/lucy/PortfolioOptimizer/')
from PortfolioOptimizerMaster.src import tools

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
high_post =pd.read_csv('/home/hengtian/kate/signal/data/quant_data/highest_post.csv',index_col=0)

In [3]:
high_post = high_post.replace(0,np.nan)
# high_post.to_csv('highest_post.csv')

In [4]:
low_post = pd.read_csv('/home/hengtian/kate/signal/data/quant_data/lowest_post.csv',index_col=0)

In [5]:
low_post = low_post.replace(0,np.nan)
# low_post.to_csv('lowest_post.csv')

In [6]:
close_price = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/close_price_post.csv",index_col = 0)
close_prie= close_price.replace(0,np.nan)

In [7]:
close_price = close_price[:'2021-07-13']

In [8]:
ovlp = high_post.columns.intersection(close_price.columns)

In [9]:
low_post = low_post[ovlp]
low_post.index=close_price.index

In [10]:
high_post = high_post[ovlp]
close_price = close_price[ovlp]

In [11]:
low_post=low_post.astype(float)

In [12]:
#涨跌幅
n_pct = close_price/close_price.shift(1)-1

In [ ]:
n_pct.to_csv('chg_pct.csv')

In [15]:
n_pct =pd.read_csv('/home/hengtian/kate/signal/feature_data/chg_pct.csv',index_col=0)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [16]:
high_post=high_post.astype(float)
high_post.index=close_price.index

In [ ]:
#最高涨幅
max_pct =high_post/close_price.shift(1)-1

In [ ]:
max_pct.to_csv('max_rise.csv')

In [17]:
max_pct =pd.read_csv('/home/hengtian/kate/signal/feature_data/max_rise.csv',index_col=0)

In [36]:
#40日avg
forty_avg = close_price.rolling(40,min_periods =20).mean()

In [37]:
#收盘价大于40日均线
close_vs_forty= close_price/forty_avg.shift(1)

In [38]:
close_vs_forty.to_csv('close_vs_40mean.csv')

In [18]:
close_vs_forty = pd.read_csv('/home/hengtian/kate/signal/feature_data/close_vs_40mean.csv',index_col=0)

In [ ]:
# 当天最高价/20日内最低价
min_20 = low_post.rolling(20,min_periods = 10).min()
high_vs_20 = high_post/min_20.shift(1)

In [60]:
low_post

,000001.XSHE,000002.XSHE,000003.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,...,688699.XSHG,688700.XSHG,688777.XSHG,688788.XSHG,688789.XSHG,688793.XSHG,688799.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
2013-01-04,661.769,1064.505,NaN,55.492,28.692,256.294,114.052,32.051,38.318,50.371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-07,660.106,1064.505,NaN,55.492,28.594,252.110,111.565,31.352,38.274,50.371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-08,659.275,1064.505,NaN,56.038,28.594,262.048,112.528,32.416,38.010,50.371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-09,656.781,1064.505,NaN,56.175,28.790,258.910,120.308,32.689,38.406,50.371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-10,656.365,1064.505,NaN,56.038,28.889,261.002,121.511,33.145,38.846,50.371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-07,2891.940,3372.835,NaN,124.720,18.178,333.574,51.973,58.364,177.294,42.021,...,316.441,59.60,98.715,130.244,NaN,NaN,NaN,44.739,56.50,74.00
2021-07-08,2724.850,3336.250,NaN,124.788,18.473,329.422,55.702,57.826,191.506,41.913,...,332.051,61.31,98.404,127.529,150.00,NaN,NaN,45.977,57.47,74.50
2021-07-09,2701.715,3351.728,NaN,126.291,19.063,327.346,54.018,57.826,204.307,41.805,...,323.654,57.96,99.287,127.368,156.11,NaN,NaN,44.202,56.69,74.08
2021-07-12,2685.006,3346.100,NaN,129.025,18.768,332.882,55.341,58.633,222.752,42.021,...,329.510,58.88,98.956,129.741,171.11,NaN,NaN,46.272,55.68,74.40


In [87]:
i = 20
while i<len(low_post):
    temp =low_post.iloc[i-20:i,:]
    m_d = []
    for col in temp.columns:
#         print(temp[col])
        idx = np.argmin(temp[col])
        m_d.append(temp[col].index[idx])
    date =low_post.index[i]
#     print(date)
    m_df=pd.DataFrame(m_d,columns = temp.columns)
    print(m_df)
    break
    i+=1
    
def find_min_date(stock,q):
    i = 20
    temp = low_post[stock]
    min_d = []
    while i<len(temp):
        if len(temp[i-20:i,:].dropna())<10:
            i+=1
        else:
            idx =np.argmin(temp[i-20:i,:])
            min_d.append(temp.index[idx])

ValueError: Shape of passed values is (4536, 1), indices imply (4536, 4536)

In [63]:
def min_index(x):
    return x.min().index

In [ ]:
zz_highest = pd.read_csv('/home/hengtian/kate/signal/data/highest_index.csv',index_col = 0)
zz_lowest = pd.read_csv('/home/hengtian/kate/signal/data/lowest_index.csv',index_col = 0)

In [ ]:
high_vs_20.to_csv('highest_vs_past20.csv')

In [19]:
high_vs_20 = pd.read_csv('/home/hengtian/kate/signal/feature_data/highest_vs_past20.csv',index_col=0)

In [ ]:
#当日和前5日均量
amt = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/turnover_value.csv",index_col = 0)
amt = amt['2013':'2021-07-13']
amt = amt[ovlp]
amt= amt.replace(0,np.nan)
amt = amt.shift(-1)
amt_5_avg = amt.rolling(5,min_periods=3).mean()
amt_past5 = amt/amt_5_avg.shift(1)

In [ ]:
amt_past5.to_csv('amt_past5avg.csv')

In [20]:
amt_past5 = pd.read_csv('/home/hengtian/kate/signal/feature_data/amt_past5avg.csv',index_col=0)

In [ ]:
# 60日均成交金额
tov_60 = amt.rolling(60,min_periods = 30).mean()

In [ ]:
tov_60.to_csv('tov_60.csv')

In [21]:
tov_60 = pd.read_csv('/home/hengtian/kate/signal/feature_data/tov_60.csv',index_col=0)

In [ ]:
#100天前的700日内最高价/100日内最低价
max_700 = high_post.rolling(700,min_periods =300).max()
min_100 = low_post.rolling(100,min_periods = 50).min()
max700_vs_min100 = max_700.shift(100)/min_100

In [ ]:
max700_vs_min100.to_csv('max700_vs_min100.csv')

In [22]:
max700_vs_min100 = pd.read_csv('/home/hengtian/kate/signal/feature_data/max700_vs_min100.csv',index_col=0)

In [39]:
#12日内最低价vs100日内最低价
min_12 = low_post.rolling(12,min_periods=6).min()
min_100 = low_post.rolling(100,min_periods =50).min()
min_12_100 = min_12/min_100

In [40]:
min_12_100.to_csv('min12_vs_min100.csv')

In [23]:
min_12_100 = pd.read_csv('/home/hengtian/kate/signal/feature_data/min12_vs_min100.csv',index_col=0)

# Merge

In [43]:
comp_dict ={
    'max_pct':max_pct,
    'close_vs_forty':close_vs_forty,
    'high_vs_20':high_vs_20,
    'amt_past5':amt_past5,
    'tov_60':tov_60,
    'max700_vs_min100':max700_vs_min100,
    'min_12_100':min_12_100
}

In [44]:
og = n_pct.unstack().rename('n_pct')
df = pd.DataFrame(og)
df.index.names = ['stock_code','date']
for comp_name,comp_df in comp_dict.items():
    temp =comp_df.unstack().rename(comp_name)
    temp.index.names = ['stock_code','date']
    df = pd.merge(df,temp.reset_index(),on = ['stock_code','date'])

In [46]:
df.sort_values('min_12_100')

,stock_code,date,n_pct,max_pct,close_vs_forty,high_vs_20,amt_past5,tov_60,max700_vs_min100,min_12_100
4804928,300801.XSHE,2021-05-18,-0.007806,-0.002783,0.904435,1.030514,0.867175,3.800031e+07,NaN,1.0
8300142,603787.XSHG,2020-05-07,-0.020206,0.005543,0.890198,1.137209,0.567521,6.647871e+07,3.123357,1.0
8300143,603787.XSHG,2020-05-08,0.100374,0.100374,0.988631,1.219312,1.742677,6.574289e+07,3.123357,1.0
8300144,603787.XSHG,2020-05-11,0.048594,0.087735,1.041830,1.326289,3.233124,6.671391e+07,3.123357,1.0
8300145,603787.XSHG,2020-05-12,-0.008146,0.003242,1.038290,1.282710,1.025494,6.663588e+07,3.123357,1.0
...,...,...,...,...,...,...,...,...,...,...
9398463,689009.XSHG,2020-12-30,0.044539,0.077847,1.133539,1.309647,0.944858,8.768734e+08,NaN,NaN
9398464,689009.XSHG,2020-12-31,0.060685,0.087628,1.191266,1.380392,0.952593,8.695970e+08,NaN,NaN
9398465,689009.XSHG,2021-01-04,0.027732,0.077837,1.213848,1.447801,1.042568,8.636247e+08,NaN,NaN
9398466,689009.XSHG,2021-01-05,-0.009070,0.013946,1.194781,1.399750,1.034050,8.578177e+08,NaN,NaN


In [ ]:
df = df.set_index(['stock_code','date'])

In [ ]:
cp1 = df.reset_index()

In [ ]:
cp1[(cp1.stock_code =='601155.XSHG')&(cp1.date =='2018-11-19')]

In [31]:
sys.path.append(r'/home/hengtian/lucy/get_factor_tools/')
import add_factor_tools as aft

In [29]:
copy =df.copy()

In [47]:
ft=aft.add_ipo_date(df,'date')

In [48]:
ft = ft.drop('ipo_date',axis = 1)

In [34]:
ft = ft.set_index(['stock_code','date'])

In [49]:
ft.to_csv('feature_selection_v4.csv')

In [13]:
ft =pd.read_csv('/home/hengtian/kate/signal/feature_selection/feature_selection_v4.csv',index_col =0)
ft

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,stock_code,date,n_pct,max_pct,close_vs_forty,high_vs_20,amt_past5,tov_60,max700_vs_min100,min_12_100,time_f_ipo
0,000001.XSHE,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7947
1,000001.XSHE,2013-01-07,0.019387,0.022513,NaN,NaN,NaN,NaN,NaN,NaN,7950
2,000001.XSHE,2013-01-08,-0.018406,0.004295,NaN,NaN,NaN,NaN,NaN,NaN,7951
3,000001.XSHE,2013-01-09,-0.008749,0.001251,NaN,NaN,0.667132,NaN,NaN,NaN,7952
4,000001.XSHE,2013-01-10,0.000629,0.015132,NaN,NaN,0.697924,NaN,NaN,NaN,7953
...,...,...,...,...,...,...,...,...,...,...,...
9398587,689009.XSHG,2021-07-07,0.035966,0.038631,1.022617,1.058656,0.735744,1.425459e+08,NaN,1.269828,251
9398588,689009.XSHG,2021-07-08,-0.027260,-0.001029,0.992206,1.054854,1.065150,1.423447e+08,NaN,1.269828,252
9398589,689009.XSHG,2021-07-09,-0.009914,0.012822,0.980716,1.040326,0.612809,1.416658e+08,NaN,1.269828,253
9398590,689009.XSHG,2021-07-12,0.005340,0.020561,0.983253,1.037882,0.836393,1.424033e+08,NaN,1.269828,256


In [14]:
f1 = ft[(ft['n_pct']>0.02) & (ft['max_pct']>0.03)]

In [15]:
f2= f1[(f1['close_vs_forty']>1) & (f1['high_vs_20']>1.2)&(f1['high_vs_20']<1.6)]

In [16]:
f3 = f2[(f2['tov_60']>3e+07)&(f2['max700_vs_min100']>1.5)&(f2['amt_past5']>0.6)]

In [17]:
f4 = f3[(f3['min_12_100']>1)&(f3['time_f_ipo']>300)]

In [18]:
f4.to_csv('feature_selected_v7.csv')

In [ ]:
c4 = f4.reset_index()
c4[(c4.stock_code =='601155.XSHG')&(c4.date=='2018-11-19')]

In [ ]:
type(datetime(2013,1,1))

In [24]:
f4[f4.date<'2014']

,stock_code,date,n_pct,max_pct,close_vs_forty,high_vs_20,amt_past5,tov_60,max700_vs_min100,min_12_100,time_f_ipo
